##Imports

In [ ]:
 import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
if tf.__version__ != "2.4.1":
    !pip3 install tensorflow==2.4.1
    import os
    os._exit(00)

##Utils & Visualization



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
base_dir = 'drive/MyDrive/FireSage_Colab/dataset'
train_dir = 'drive/MyDrive/FireSage_Colab/datasets/train'
val_dir = 'drive/MyDrive/FireSage_Colab/datasets/validation'
 
train_dataGen = ImageDataGenerator(rescale=1./255)
val_dataGen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_dataGen.flow_from_directory(train_dir, 
                                                    target_size=(224,224),
                                                    batch_size=50,
                                                    class_mode='binary')
 
val_generator = val_dataGen.flow_from_directory(val_dir, target_size=(224,224), 
                                                batch_size=50,
                                                class_mode='binary')

Found 10164 images belonging to 2 classes.
Found 3100 images belonging to 2 classes.


In [ ]:
import matplotlib.pyplot as plt
CLASSES = ["Fire", "Normal"]
for data_batch, labels_batch in train_generator:
 
    print('data batch shape:', data_batch.shape)
 
    print('labels batch shape:', labels_batch)
    break
 
fig = plt.figure(figsize=(20,20), dpi=200)
# x, y = data_batch.next()
for i in range(0,50):
    ax = fig.add_subplot(10, 5, i+1)
    image = data_batch[i]
    ax.set_title(CLASSES [int(labels_batch[i])])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(image)

In [ ]:
   def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        #plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

##Networks

###SqueezeNet

In [ ]:
prob=0.2
bnmomemtum=0.9
IMAGE_SIZE=[224, 224] 
def fire(x, squeeze, expand):
    y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
    y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
    y1 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y1)
    y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
    y3 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y3)
    return tf.keras.layers.concatenate([y1, y3])
 
def fire_module(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)
 
x_bytes = tf.keras.layers.Input(shape=[*IMAGE_SIZE, 3]) # input is 192x192 pixels RGB
 
y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x_bytes)
y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = tf.keras.layers.Dropout(prob)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = tf.keras.layers.Dropout(prob)(y)
y = fire_module(64, 128)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = tf.keras.layers.Dropout(prob)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = tf.keras.layers.Dropout(prob)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y_bytes = tf.keras.layers.Dense(1, activation='sigmoid')(y)
 
model = tf.keras.Model(x_bytes, y_bytes)
 
model.compile(
    optimizer='adam',
    loss= 'binary_crossentropy',
    metrics=['accuracy'])
def build():
     return model
 # model.summary()

###Xception

In [ ]:
def create_model():
    IMAGE_SIZE=[224, 224]
    
    #pretrained_model = tf.keras.applications.MobileNetV2(input_shape=[*IMAGE_SIZE, 3], include_top=False)
    pretrained_model = tf.keras.applications.Xception(input_shape=[*IMAGE_SIZE, 3], include_top=False)
    #pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    pretrained_model.trainable = True
 
    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        #tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
 
    model.compile(
        optimizer='adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy']
    )
 
    return model

###Training

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard, LearningRateScheduler
 
base_dir = 'drive/MyDrive/FireSage_Colab/datasets'
train_dir = 'drive/MyDrive/FireSage_Colab/datasets/train'
val_dir = 'drive/MyDrive/FireSage_Colab/datasets/validation'
test_dir = 'drive/MyDrive/FireSage_Colab/datasets/test'
file_path = 'drive/MyDrive/FireSage_Colab/checkpoints-SqueezeNet-0/weights-improvement-{epoch:02d}-{val_accuracy:.2f}'
log_dir = 'drive/MyDrive/FireSage_Colab/tesorboard_log_dir'
 
# BATCH_SIZE = 16
# VALIDATION_BATCH_SIZE = 16
start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.0002
rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8
 
train_dataGen = ImageDataGenerator(
                                   rescale=1./255,
                                   width_shift_range=[-200,200],
                                   horizontal_flip=True,
                                   zoom_range=[0.5,1.0],
                                   shear_range=2.0,
                                   brightness_range=[1.0,1.5],
                                   fill_mode='nearest'
                                   )
val_dataGen = ImageDataGenerator(rescale=1./255,)                                  
 
train_generator = train_dataGen.flow_from_directory(train_dir, target_size=(224,224), batch_size=64, shuffle=True,
                                                    classes=["Normal", "Fire"], class_mode='binary')
val_generator = val_dataGen.flow_from_directory(val_dir, target_size=(224,224), batch_size=32, shuffle=True, classes=["Normal", "Fire"], class_mode='binary')
                                                             
def lrfn(epoch):
    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        if epoch < rampup_epochs:
            lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        else:
            lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
        return lr
    return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)
                                             
# new_model= SqueezeNet(1)
new_model = build()
#new_model = tf.keras.models.load_model('drive/MyDrive/FireSage_Colab/checkpoints-SqueezeNet-0/weights-improvement-01-0.82')
#new_model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])
   
callback_list = [
    ModelCheckpoint(
        file_path,
        monitor='val_accuracy',
        save_freq='epoch',
        ),
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.1,
        patience=3,
    ),
     # EarlyStopping(
     #   patience=3,
     #   mode='max',
     #   monitor='val_accuracy'
     #),
    #LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True),
    TensorBoard(
        log_dir='drive/MyDrive/FireSage_Colab/tb_logs-1'
    )
]
 
 
history=new_model.fit(
            train_generator,
            steps_per_epoch=78,
            epochs=25,
            callbacks=callback_list,
            validation_data=val_generator,
            validation_steps=10)
 
 
 
new_model.save('drive/MyDrive/FireSage_Colab/saved_models/FireSage-SqueezeNet-2')

71/78 [==========================>...] - ETA: 9s - loss: 0.4892 - accuracy: 0.7671 

In [ ]:
 print(history.history.keys())
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)

In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model('drive/MyDrive/FireSage_Colab/SqueezeNet-checkpoints-0/weights-improvement-18-0.95')
new_model.summary()

###Tensorboard

In [ ]:
 %load_ext tensorboard

In [ ]:
 %tensorboard --logdir 'drive/MyDrive/FireSage_Colab/tb_logs'

###Evaluation & Testing

In [ ]:
new_model = tf.keras.models.load_model('drive/MyDrive/FireSage_Colab/checkpoints-Xception-0/weights-improvement-05-0.97')
new_model.save('drive/MyDrive/FireSage_Colab/FireSage-Xception-0.97.h5')

In [ ]:
import os
import shutil
from PIL import Image
import tensorflow as tf
from PIL import Image
import numpy as np
from skimage import transform
 
M = []
 
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image
filenames = os.listdir('drive/MyDrive/FireSage_Colab/tmp')
for i in filenames:
  image = load(f'drive/MyDrive/FireSage_Colab/tmp/{i}')
  pred = new_model.predict(image)
  M.append(pred)
  # print(f'the fire probabilty is {pred[0][0]*100:.2f}%')
arg=''' 
Avg=[]
i=0
path="drive/MyDrive/FireSage_Colab/datasets/train/Fire"
 drive/MyDrive/FireSage_Colab/train_samples/Fire
for dirpath, dirnames, files in os.walk(path):
  for file_name in files:
    abs_file = os.path.join(path, file_name)
    image = load(abs_file)
    pred = new_model.predict(image)
    if pred[0][0] <= 0.1999:
      print(file_name, pred[0][0])
      shutil.move(abs_file, 'drive/MyDrive/FireSage_Colab/train_samples/Fire/')
      i+=1
      Avg.append(pred[0][0])
Average=sum(Avg)/len(Avg)
print(f"found {i} files. Average prediction is {Average}")
'''

In [7]:
 from google.colab import auth
auth.authenticate_user()

In [8]:
!gcloud config set project custom-hold-311317

Updated property [core/project].


In [9]:
!gsutil cp -r drive/MyDrive/FireSage_Colab/checkpoints-Xception-0/weights-improvement-05-0.97 gs://fire-temple_77/

Copying file://drive/MyDrive/FireSage_Colab/checkpoints-Xception-0/weights-improvement-05-0.97/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://drive/MyDrive/FireSage_Colab/checkpoints-Xception-0/weights-improvement-05-0.97/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite obj

In [ ]:
from google.cloud import storage

storage_client = storage.Client.from_service_account_json("drive/MyDrive/FireSage_Colab/custom-hold-311317-aa296314ddd8.json")
bucket = storage_client.bucket("fire-temple_77")

blob_name = "FireSage-Xception-0.97.h5"
blob = bucket.blob(blob_name)
blob.upload_from_filename('drive/MyDrive/FireSage_Colab/FireSage-Xception-0.97.h5')
print(f"uploaded {blob_name} to {blob_name}")

uploaded FireSage-Xception-0.97.h5 to FireSage-Xception-0.97.h5


Model.evaluate()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
test_dir = 'drive/MyDrive/FireSage_Colab/datasets/test'
 
test_dataGen = ImageDataGenerator(rescale=1./255)
 
test_generator = test_dataGen.flow_from_directory(test_dir, 
                                                    target_size=(224,224),
                                                    batch_size=100,
                                                    classes=["normal", "Fire"],
                                                    class_mode='binary')

Found 0 images belonging to 2 classes.


In [ ]:
loss, acc = new_model.evaluate(test_generator, verbose=1)
print(f'{loss:.3f}, {acc*100:4.2f}%')

13/13 [==============================] - 638s 53s/step - loss: 0.4081 - accuracy: 0.8587
0.408, 85.87%


###Data Sourcing & Wrangling

In [ ]:
# https://www.floydhub.com/api/v1/resources/Ad75bXjK3mdBC4P9wbsEYn?content=true&download=true&rename=hackai21g2-datasets-fire-1
# https://www.floydhub.com/api/v1/resources/q2nxxfijyZ5ZCCGTWDNMwN?content=true&download=true&rename=ninaduf-datasets-fire-1
# https://www.floydhub.com/api/v1/resources/RwGobMM8FhzX9sUcgnBJTa?content=true&download=true&rename=ninaduf-datasets-no_fire-1
# drive/MyDrive/FireSage_Colab/derf1certif-datasets-f.tar

In [ ]:
!ls "drive/MyDrive/FireSage_Colab"

 datasets	  First_Colab.ipynb   Titanic.csv
 FireSage.ipynb   mnist_cnn.py	     'Untitled (1).ipynb'


In [ ]:
import requests
 
print('Beginning file download with requests')
 
url = 'https://drive.google.com/file/d/11KBgD_W2yOxhJnUMiyBkBzXDPXhVmvCt/view?usp=drivesdk'
r = requests.get(url)
 
with open('drive/MyDrive/FireSage_Colab/derf1certif-datasets-f.tar', 'wb') as f:
    f.write(r.content)
 
# Retrieve HTTP meta-data
print(r.status_code)
print(r.headers['content-type'])
print(r.encoding)

Beginning file download with requests
200
text/html; charset=utf-8
utf-8


In [ ]:
import os
import tarfile
 
data_path = 'drive/MyDrive/FireSage_Colab/datasets/derf1fire'
data_file = 'drive/MyDrive/FireSage_Colab/derf1certif-datasets-f.tar'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    with tarfile.open(data_file) as tar:
        tar.extractall(path=data_path)

In [ ]:
!cd drive/MyDrive/FireSage_Colab/datasets/block-4
!find | wc -l

6828


In [ ]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('drive/MyDrive/Training Dataset.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('drive/MyDrive/FireSage_Colab')

In [ ]:
import os
# List all files in a directory using os.listdir
 
basepath = 'drive/MyDrive/FireSage_Colab/datasets/train/Fire'
entries = os.listdir(basepath)
#if os.path.isfile(os.path.join(basepath, entries)):
print(len(entries))

4258


In [ ]:
!mv drive/MyDrive/FireSage_Colab/train_samples/Fire/* drive/MyDrive/FireSage_Colab/datasets/train/Normal/

In [ ]:
import os
import shutil
f = open('drive/MyDrive/prune.sh', 'r')
for line in f:
   l = str(line).split("/")
   fname=l[2].strip("\n")
   os.remove(f"drive/MyDrive/FireSage_Colab/Robbery_Accident_Fire_Database2/Fire/{fname}")
   #print(fname)

In [ ]:
import os
import shutil
from PIL import Image
 
L=[]
path="drive/MyDrive/FireSage_Colab/train_samples/Fire"
i=0
for dirpath, dirnames, files in os.walk(path):
  for file_name in files:
     abs_file = os.path.join(path, file_name)
     shutil.copy(abs_file, 'drive/MyDrive/FireSage_Colab/datasets/train/Fire/')
     #img=Image.open(abs_file)
     #if not img.mode in L:
     #     L.append(img.mode)
     #if file_name[-3:]  == "png":
     #if img.mode in ("L", "P"):
     #    print(i, file_name[-3:], img.mode)
     #   os.remove(abs_file)
     #if i > 10:
      #  break
     i+=1

In [ ]:
!cd drive/MyDrive
!git clone https://github.com/AllenDowney/ThinkDSP.git

Cloning into 'ThinkDSP'...
remote: Enumerating objects: 2421, done.
remote: Total 2421 (delta 0), reused 0 (delta 0), pack-reused 2421
Receiving objects: 100% (2421/2421), 207.80 MiB | 29.98 MiB/s, done.
Resolving deltas: 100% (1320/1320), done.


In [ ]:
!cd drive/MyDrive
!ls

drive  sample_data  ThinkDSP
